In [2]:
dir

<function dir>

In [3]:
pwd

'/Users/pichappan/Documents/SamplePython/notebook'

In [4]:
cd ../input

/Users/pichappan/Documents/SamplePython/input


In [5]:
import pandas as pd
import numpy as np
from sklearn import linear_model

In [6]:
dft = pd.read_csv('http://bit.ly/kaggletrain')
df1 = pd.read_csv("Airbnb_NYC.csv")
df1.tail()

,Boroughs,Latitude,Longitude,Prop_Type,Min_Nights,Host_Listing_Cnt,Days_Available,Review_Cnt,Reviews30d,Price
38728,Staten Island,40.64641,-74.08502,EntireHome,2.0,1.0,106,58,4.73,130.0
38729,Staten Island,40.64682,-74.08734,PrivateRoom,20.0,1.0,89,58,1.05,43.0
38730,Staten Island,40.64730,-74.08408,PrivateRoom,30.0,4.0,0,85,1.35,105.0
38731,Staten Island,40.64762,-74.08682,EntireHome,3.0,1.0,232,5,1.88,135.0
38732,Staten Island,40.64779,-74.08460,EntireHome,1.0,1.0,250,108,2.32,140.0


In [7]:
# Data Preprocessing
df1 = df1.dropna(thresh=9)
df1.Min_Nights.fillna(df1.Min_Nights.median())
df1.Host_Listing_Cnt.fillna(df1.Host_Listing_Cnt.median())
df1.Days_Available.fillna(df1.Days_Available.median())
df1.Review_Cnt.fillna(df1.Review_Cnt.median())
df1.Reviews30d.fillna(df1.Reviews30d.median())
df1.Price.fillna(df1.Price.median())
df1.Prop_Type.value_counts()

EntireHome     20173
PrivateRoom    17720
SharedRoom       840
Name: Prop_Type, dtype: int64

In [8]:
# New Column with map method is quick and easy
df1['isEntireHome'] = df1.Prop_Type.map({'EntireHome':1,'PrivateRoom':0,'SharedRoom':0})
df1['isPrivateRoom'] = df1.Prop_Type.map({'EntireHome':0,'PrivateRoom':1,'SharedRoom':0})
df1

,Boroughs,Latitude,Longitude,Prop_Type,Min_Nights,Host_Listing_Cnt,Days_Available,Review_Cnt,Reviews30d,Price,isEntireHome,isPrivateRoom
0,Bronx,40.80011,-73.91330,PrivateRoom,21.0,2.0,291,19,0.27,60.0,0,1
1,Bronx,40.80024,-73.91422,PrivateRoom,3.0,1.0,0,59,2.93,49.0,0,1
2,Bronx,40.80074,-73.91574,PrivateRoom,3.0,1.0,354,3,0.87,50.0,0,1
3,Bronx,40.80091,-73.91449,PrivateRoom,21.0,2.0,117,18,0.36,71.0,0,1
4,Bronx,40.80143,-73.91353,EntireHome,2.0,3.0,0,12,3.50,115.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
38728,Staten Island,40.64641,-74.08502,EntireHome,2.0,1.0,106,58,4.73,130.0,1,0
38729,Staten Island,40.64682,-74.08734,PrivateRoom,20.0,1.0,89,58,1.05,43.0,0,1
38730,Staten Island,40.64730,-74.08408,PrivateRoom,30.0,4.0,0,85,1.35,105.0,0,1
38731,Staten Island,40.64762,-74.08682,EntireHome,3.0,1.0,232,5,1.88,135.0,1,0


In [9]:
# One Hot Encoding , Dummy Variables. This is flexable 
dummyBoro = pd.get_dummies(df1.Boroughs, prefix='is').iloc[:,:4] # [all rows, 4 column from start] dropped last column
dummyProp = pd.get_dummies(df1.Prop_Type, prefix='is').iloc[:,1:] # [all rows, column 1 through end] dropped first column

In [10]:
# df2 = pd.concat([df1,dummyBoro, dummyProp], axis='columns') # concat 3 df
df2 = pd.concat([df1,dummyBoro], axis='columns') # or axis=1
df2.head()

,Boroughs,Latitude,Longitude,Prop_Type,Min_Nights,Host_Listing_Cnt,Days_Available,Review_Cnt,Reviews30d,Price,isEntireHome,isPrivateRoom,is_Bronx,is_Brooklyn,is_Manhattan,is_Queens
0,Bronx,40.80011,-73.91330,PrivateRoom,21.0,2.0,291,19,0.27,60.0,0,1,1,0,0,0
1,Bronx,40.80024,-73.91422,PrivateRoom,3.0,1.0,0,59,2.93,49.0,0,1,1,0,0,0
2,Bronx,40.80074,-73.91574,PrivateRoom,3.0,1.0,354,3,0.87,50.0,0,1,1,0,0,0
3,Bronx,40.80091,-73.91449,PrivateRoom,21.0,2.0,117,18,0.36,71.0,0,1,1,0,0,0
4,Bronx,40.80143,-73.91353,EntireHome,2.0,3.0,0,12,3.50,115.0,1,0,1,0,0,0


In [11]:
# Dummy variable Trap. Multi Colinear variables. Now of catagorical variables
df3 = df2.drop(['Boroughs','Prop_Type'],axis='columns')
df3.dropna(inplace=True) # Drop rows with NaN
df3.shape

(38331, 14)

In [12]:
# is all this too hard? don't worry we have a quick and easy way. Pass df instand of Series to pd.get_dummies
df2=pd.get_dummies(df1, columns=['Boroughs','Prop_Type'], drop_first=True)
df2

,Latitude,Longitude,Min_Nights,Host_Listing_Cnt,Days_Available,Review_Cnt,Reviews30d,Price,isEntireHome,isPrivateRoom,Boroughs_Brooklyn,Boroughs_Manhattan,Boroughs_Queens,Boroughs_Staten Island,Prop_Type_PrivateRoom,Prop_Type_SharedRoom
0,40.80011,-73.91330,21.0,2.0,291,19,0.27,60.0,0,1,0,0,0,0,1,0
1,40.80024,-73.91422,3.0,1.0,0,59,2.93,49.0,0,1,0,0,0,0,1,0
2,40.80074,-73.91574,3.0,1.0,354,3,0.87,50.0,0,1,0,0,0,0,1,0
3,40.80091,-73.91449,21.0,2.0,117,18,0.36,71.0,0,1,0,0,0,0,1,0
4,40.80143,-73.91353,2.0,3.0,0,12,3.50,115.0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38728,40.64641,-74.08502,2.0,1.0,106,58,4.73,130.0,1,0,0,0,0,1,0,0
38729,40.64682,-74.08734,20.0,1.0,89,58,1.05,43.0,0,1,0,0,0,1,1,0
38730,40.64730,-74.08408,30.0,4.0,0,85,1.35,105.0,0,1,0,0,0,1,1,0
38731,40.64762,-74.08682,3.0,1.0,232,5,1.88,135.0,1,0,0,0,0,1,0,0


In [13]:
#Class Object
lrModel = linear_model.LinearRegression()

In [14]:
X = df3.drop('Price',axis='columns')
X

,Latitude,Longitude,Min_Nights,Host_Listing_Cnt,Days_Available,Review_Cnt,Reviews30d,isEntireHome,isPrivateRoom,is_Bronx,is_Brooklyn,is_Manhattan,is_Queens
0,40.80011,-73.91330,21.0,2.0,291,19,0.27,0,1,1,0,0,0
1,40.80024,-73.91422,3.0,1.0,0,59,2.93,0,1,1,0,0,0
2,40.80074,-73.91574,3.0,1.0,354,3,0.87,0,1,1,0,0,0
3,40.80091,-73.91449,21.0,2.0,117,18,0.36,0,1,1,0,0,0
4,40.80143,-73.91353,2.0,3.0,0,12,3.50,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38728,40.64641,-74.08502,2.0,1.0,106,58,4.73,1,0,0,0,0,0
38729,40.64682,-74.08734,20.0,1.0,89,58,1.05,0,1,0,0,0,0
38730,40.64730,-74.08408,30.0,4.0,0,85,1.35,0,1,0,0,0,0
38731,40.64762,-74.08682,3.0,1.0,232,5,1.88,1,0,0,0,0,0


In [15]:
y = df3.Price
y

0         60.0
1         49.0
2         50.0
3         71.0
4        115.0
         ...  
38728    130.0
38729     43.0
38730    105.0
38731    135.0
38732    140.0
Name: Price, Length: 38331, dtype: float64

In [16]:
from sklearn.model_selection import train_test_split


In [17]:
#same output when random_state
Xlearn, Xtest, yLearn, yTest = train_test_split(X, y, test_size=0.35, random_state=10)

In [18]:
Xtest.shape

(13416, 13)

In [19]:
Xtest

,Latitude,Longitude,Min_Nights,Host_Listing_Cnt,Days_Available,Review_Cnt,Reviews30d,isEntireHome,isPrivateRoom,is_Bronx,is_Brooklyn,is_Manhattan,is_Queens
1882,40.63902,-74.01837,30.0,1.0,217,4,0.52,1,0,0,1,0,0
21525,40.73208,-74.00784,2.0,1.0,0,1,0.03,1,0,0,0,1,0
25861,40.76295,-73.99432,30.0,31.0,299,6,0.15,1,0,0,0,1,0
22838,40.74175,-73.98135,2.0,1.0,222,5,0.68,1,0,0,0,1,0
33195,40.84656,-73.94143,1.0,3.0,212,41,0.91,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6748,40.68012,-73.97847,2.0,4.0,276,23,0.24,0,1,0,1,0,0
8628,40.68583,-73.95870,4.0,1.0,87,19,0.87,1,0,0,1,0,0
28640,40.78676,-73.95074,2.0,2.0,132,2,2.00,0,1,0,0,1,0
9751,40.68878,-73.94103,2.0,1.0,191,28,4.72,1,0,0,1,0,0


In [20]:
Xlearn.shape

(24915, 13)

In [21]:
# Training or Learning
lrModel.fit(Xlearn,yLearn)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [22]:
lrModel.intercept_

-28336.07834092804

In [27]:
lrModel.coef_

array([-1.21754495e+02, -4.48949155e+02, -5.32995017e-01, -8.09650482e-01,
        1.77431518e-01, -1.85881143e-01, -3.06857242e-01,  1.35567926e+02,
        3.41658627e+01,  1.30327556e+02,  1.18894411e+02,  1.65337959e+02,
        1.41156135e+02])

In [34]:
print('Y-Intercept: '+ str(lrModel.intercept_) + '\n\nX-Cofficient: \n' + str(lrModel.coef_))

Y-Intercept: -28336.07834092804

X-Cofficient: 
[-1.21754495e+02 -4.48949155e+02 -5.32995017e-01 -8.09650482e-01
  1.77431518e-01 -1.85881143e-01 -3.06857242e-01  1.35567926e+02
  3.41658627e+01  1.30327556e+02  1.18894411e+02  1.65337959e+02
  1.41156135e+02]


In [23]:
lrModel.predict([[40.80024, -73.91422, 13.0, 1.0, 0, 29, 2.93, 1, 0, 0, 0, 1,0]])

array([166.91332009])

In [24]:
lrModel.score(Xtest,yTest)

0.1092525543940559

In [36]:
yPred  = lrModel.predict(Xtest)
yPred

array([221.68536581, 229.20022959, 232.24429622, ..., 117.32655351,
       185.46582762, 159.6579833 ])

In [26]:
yTest

1882     100.0
21525    225.0
25861    130.0
22838    300.0
33195    199.0
         ...  
6748     100.0
8628     265.0
28640    100.0
9751     149.0
17140    240.0
Name: Price, Length: 13416, dtype: float64

In [47]:
from sklearn.metrics import mean_squared_error, r2_score
print ('Mean Square error is: %.12f' % mean_squared_error(yTest, yPred))
print (mean_squared_error(yTest, yPred))

print ('Linear Regression Score is: %.12f' + str(lrModel.score(Xtest, yTest)))
print ('Cofficient of Dermination: %.12f' % r2_score(yTest, yPred))

Mean Square error is: 33149.480491651404
33149.480491651404
Linear Regression Score is: %.12f0.1092525543940559
Cofficient of Dermination: 0.109252554394
